In [1]:
from pyspark.sql import SparkSession
from  pyspark.sql.functions import abs, round, avg, count, year
import pandas as pd
import matplotlib

# necessary imports

In [6]:
# Load spark session and read data into spark dataframe
spark = SparkSession.builder.master('local').getOrCreate()

# All the columns should be floats except for the 'Date' and 'Currency' columns.
schema = "Date date, Open float, High float, Low float, Close float, Volume float, Currency string"
df = spark.read.csv('data/coffee.csv', header=True, schema=schema)

# lower case all column names
df = df.toDF(*[c.lower() for c in df.columns])

df.show(5)

+----------+------+------+------+------+------+--------+
|      date|  open|  high|   low| close|volume|currency|
+----------+------+------+------+------+------+--------+
|2000-01-03|122.25| 124.0| 116.1| 116.5|6640.0|     USD|
|2000-01-04|116.25| 120.5|115.75|116.25|5492.0|     USD|
|2000-01-05| 115.0| 121.0| 115.0| 118.6|6165.0|     USD|
|2000-01-06| 119.0| 121.4| 116.5|116.85|5094.0|     USD|
|2000-01-07|117.25|117.75| 113.8|114.15|6855.0|     USD|
+----------+------+------+------+------+------+--------+
only showing top 5 rows



In [7]:
# Add a column to the DataFrame where the values are the difference between 'Open' and 'Close'.
df = df.withColumn('daily_diff', round((df['close'] - df['open']), 2))

df.show(5)

+----------+------+------+------+------+------+--------+----------+
|      date|  open|  high|   low| close|volume|currency|daily_diff|
+----------+------+------+------+------+------+--------+----------+
|2000-01-03|122.25| 124.0| 116.1| 116.5|6640.0|     USD|     -5.75|
|2000-01-04|116.25| 120.5|115.75|116.25|5492.0|     USD|       0.0|
|2000-01-05| 115.0| 121.0| 115.0| 118.6|6165.0|     USD|       3.6|
|2000-01-06| 119.0| 121.4| 116.5|116.85|5094.0|     USD|     -2.15|
|2000-01-07|117.25|117.75| 113.8|114.15|6855.0|     USD|      -3.1|
+----------+------+------+------+------+------+--------+----------+
only showing top 5 rows



In [8]:
# Add a column to the DataFrame where the values are the difference between 'High' and 'Low'.
df = df.withColumn('high_low', round((df['high']-df['low']), 2))

df.show(5)

+----------+------+------+------+------+------+--------+----------+--------+
|      date|  open|  high|   low| close|volume|currency|daily_diff|high_low|
+----------+------+------+------+------+------+--------+----------+--------+
|2000-01-03|122.25| 124.0| 116.1| 116.5|6640.0|     USD|     -5.75|     7.9|
|2000-01-04|116.25| 120.5|115.75|116.25|5492.0|     USD|       0.0|    4.75|
|2000-01-05| 115.0| 121.0| 115.0| 118.6|6165.0|     USD|       3.6|     6.0|
|2000-01-06| 119.0| 121.4| 116.5|116.85|5094.0|     USD|     -2.15|     4.9|
|2000-01-07|117.25|117.75| 113.8|114.15|6855.0|     USD|      -3.1|    3.95|
+----------+------+------+------+------+------+--------+----------+--------+
only showing top 5 rows



In [9]:
# Add a column to the DataFrame where the values are 'True' if the volume for that day was 100 or above, and otherwise 'False'.
df = df.withColumn("volume_100", (df['volume'] >= 100))

df.show(5)

+----------+------+------+------+------+------+--------+----------+--------+----------+
|      date|  open|  high|   low| close|volume|currency|daily_diff|high_low|volume_100|
+----------+------+------+------+------+------+--------+----------+--------+----------+
|2000-01-03|122.25| 124.0| 116.1| 116.5|6640.0|     USD|     -5.75|     7.9|      true|
|2000-01-04|116.25| 120.5|115.75|116.25|5492.0|     USD|       0.0|    4.75|      true|
|2000-01-05| 115.0| 121.0| 115.0| 118.6|6165.0|     USD|       3.6|     6.0|      true|
|2000-01-06| 119.0| 121.4| 116.5|116.85|5094.0|     USD|     -2.15|     4.9|      true|
|2000-01-07|117.25|117.75| 113.8|114.15|6855.0|     USD|      -3.1|    3.95|      true|
+----------+------+------+------+------+------+--------+----------+--------+----------+
only showing top 5 rows



In [10]:
# Once you have a column for the difference between 'Open' and 'Close', add another column that contains the absolute values of the numbers in that column.
df = df.withColumn('abs_diff', round(abs(df.daily_diff), 2))

df.show(5)

+----------+------+------+------+------+------+--------+----------+--------+----------+--------+
|      date|  open|  high|   low| close|volume|currency|daily_diff|high_low|volume_100|abs_diff|
+----------+------+------+------+------+------+--------+----------+--------+----------+--------+
|2000-01-03|122.25| 124.0| 116.1| 116.5|6640.0|     USD|     -5.75|     7.9|      true|    5.75|
|2000-01-04|116.25| 120.5|115.75|116.25|5492.0|     USD|       0.0|    4.75|      true|     0.0|
|2000-01-05| 115.0| 121.0| 115.0| 118.6|6165.0|     USD|       3.6|     6.0|      true|     3.6|
|2000-01-06| 119.0| 121.4| 116.5|116.85|5094.0|     USD|     -2.15|     4.9|      true|    2.15|
|2000-01-07|117.25|117.75| 113.8|114.15|6855.0|     USD|      -3.1|    3.95|      true|     3.1|
+----------+------+------+------+------+------+--------+----------+--------+----------+--------+
only showing top 5 rows



In [20]:
# Compute a column called net_sales which is the average of opening, high, low, and closing cost times the volume: net_sales = avg(opening, high, low, closing price) * volume
df = df.withColumn("net_sales", ((df.open + df.high + df.low + df.close) / 4) * df.volume)

df.show(5)

+----------+------+------+------+------+------+--------+----------+--------+----------+--------+-----------------+
|      date|  open|  high|   low| close|volume|currency|daily_diff|high_low|volume_100|abs_diff|        net_sales|
+----------+------+------+------+------+------+--------+----------+--------+----------+--------+-----------------+
|2000-01-03|122.25| 124.0| 116.1| 116.5|6640.0|     USD|     -5.75|     7.9|      true|    5.75|794891.0101318359|
|2000-01-04|116.25| 120.5|115.75|116.25|5492.0|     USD|       0.0|    4.75|      true|     0.0|        643593.75|
|2000-01-05| 115.0| 121.0| 115.0| 118.6|6165.0|     USD|       3.6|     6.0|      true|     3.6|723771.0094070435|
|2000-01-06| 119.0| 121.4| 116.5|116.85|5094.0|     USD|     -2.15|     4.9|      true|    2.15|       603320.625|
|2000-01-07|117.25|117.75| 113.8|114.15|6855.0|     USD|      -3.1|    3.95|      true|     3.1|793380.5311203003|
+----------+------+------+------+------+------+--------+----------+--------+----

In [21]:
# Find the average of the values in the column that has the absolute values of the difference between 'Open' and 'Close'.
df.select(avg('abs_diff')).show(5)

+------------------+
|     avg(abs_diff)|
+------------------+
|1.7606027500686203|
+------------------+



In [22]:
# Get the count of values where the 'Volume' was less than 100.
df.select('volume_100').where(df.volume_100 == False).count()

1638

In [24]:
# Find the average 'Open' value.
# Get the highest 'High' value.

df.agg({'open': 'avg', 'high': 'max'}).show()

+---------+-----------------+
|max(high)|        avg(open)|
+---------+-----------------+
|   306.25|126.0496775257701|
+---------+-----------------+

